# Capstone Project

This notebook will mostly be used for my Capstone Project

In [197]:
import pandas as pd
import numpy as np

import requests

from bs4 import BeautifulSoup

In [198]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [199]:
soup = BeautifulSoup(page.content, 'lxml')
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [200]:
table = soup.find('table', {'class':'wikitable sortable'})

In [201]:
A=[]
B=[]
C=[]

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [202]:
df=pd.DataFrame(A, columns =['Postal Code'])
df['Borough'] = B
df['Neighbourhood'] = C

df['Postal Code'] = df['Postal Code'].str.replace('\n', '')
df['Borough'] = df['Borough'].str.replace('\n', '')
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n','')

IndexNotAssigned = df[df['Borough'] == 'Not assigned'].index

df.drop(IndexNotAssigned , inplace=True)

df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [203]:
df.shape

(103, 3)

In [204]:
df_coord = pd.read_csv('http://cocl.us/Geospatial_data')
df_coord.sort_values(by = ['Postal Code'], inplace = True)
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [205]:
column_numbers = [x for x in range(df.shape[1])]
column_numbers.remove(0)
df = df.iloc[:, column_numbers]

df.head(15)

,Borough,Neighbourhood
2,North York,Parkwoods
3,North York,Victoria Village
4,Downtown Toronto,"Regent Park, Harbourfront"
5,North York,"Lawrence Manor, Lawrence Heights"
6,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,Etobicoke,"Islington Avenue, Humber Valley Village"
9,Scarborough,"Malvern, Rouge"
11,North York,Don Mills
12,East York,"Parkview Hill, Woodbine Gardens"
13,Downtown Toronto,"Garden District, Ryerson"


In [206]:
df.Borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [207]:
!pip install geopy

In [208]:
!pip install folium

In [209]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [210]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


In [211]:
toronto_downtown_data = df[df['Borough'] == 'Downton Toronto'].reset_index(drop=True)

neighbourhood_simplified = toronto_downtown_data['Neighbourhood'].str.split(',').str[0]

toronto_downtown_data= pd.concat([toronto_downtown_data, neighbourhood_simplified],axis =1 )

column_numbers1 = [x for x in range(toronto_downtown_data.shape[1])]
column_numbers1.remove(1)
toronto_downtown_data = toronto_downtown_data.iloc[:, column_numbers1]

toronto_downtown_data.head(5)

,Borough,Neighbourhood


In [212]:
address = "Downtown Toronto"

geolocator = Nominatim(user_agent="downtown_toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [235]:
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_downtown_data['Latitude'], toronto_downtown_data['Longitude'], toronto_downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)
    
map_downtown_toronto                                                                                                                                                                                                                               

KeyError: 'Latitude'

In [234]:
neighbourhood_latitude = toronto_downtown_data.loc[0,'Latitude']
neighbourhood_longitude = toronto_downtown_data.loc[0,'Longitude']

neighbourhood_name = toronto_downtown_data.loc[0,'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name,
                                                              neighbourhood_latitude,
                                                              neighbourhood_longitude))

KeyError: 0

In [221]:
CLIENT_ID = 'XN1K2RMOXCTVQI3Q0RCXP2EF55VDKC3ZGCFGTTA15IWVNX0O'
CLIENT_SECRET = 'IAF0UZYO3CGPXG0SOST0Q0DMS4ZWZZNNCW1DORBDMRU0BSD3'
VERSION = '20180605'
LIMIT = 100

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: XN1K2RMOXCTVQI3Q0RCXP2EF55VDKC3ZGCFGTTA15IWVNX0O
CLIENT_SECRET:IAF0UZYO3CGPXG0SOST0Q0DMS4ZWZZNNCW1DORBDMRU0BSD3


In [222]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
            
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighbourhood',
                        'Neighbourhood Latitude',
                        'Neighbourhood Longitude',
                        'Venue',
                        'Venue Latitude',
                        'Venue Longitude',
                        'Venue Category']
return(nearby_venues)

NameError: name 'venues_list' is not defined

In [223]:
toronto_downtown_venues = getNearbyVenues(names=toronto_downtown_data['Neighbourhood'],
                                         latitudes=toronto_downtown_data['Latitude'],
                                         longitudes=toronto_downtown_data['Longitude'])

KeyError: 'Latitude'

In [224]:
print ('There are {} uniques categories.'.format(len(toronto_downtown_venues['Venue Category'].unique())))

TypeError: 'NoneType' object is not subscriptable

In [225]:
toronto_onehot = pd.get_dummies(toronto_downtown_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_downtown_venues['Neighbourhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

TypeError: 'NoneType' object is not subscriptable

In [226]:
toronto_onehot.shape

NameError: name 'toronto_onehot' is not defined

In [227]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

NameError: name 'toronto_onehot' is not defined

In [228]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

NameError: name 'toronto_grouped' is not defined

In [229]:
def return_most_comon_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [217]:
num_top_vnues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arrange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicator[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

AttributeError: module 'numpy' has no attribute 'arrange'

In [230]:
toronto_grouped.head()

NameError: name 'toronto_grouped' is not defined

In [231]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood',1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

NameError: name 'toronto_grouped' is not defined

In [232]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronoto_downtown_data

toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neigbourhood'), on='Neighbourhood')

toronto_merged.head()

NameError: name 'neighbourhoods_venues_sorted' is not defined

In [233]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_
merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), pars_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

SyntaxError: invalid syntax (<ipython-input-233-f54f1ba905ac>, line 10)

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [218]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shaped[1]))]]

NameError: name 'toronto_merged' is not defined

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shaped[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shaped[1]))]]

In [219]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shaped[1]))]]

NameError: name 'toronto_merged' is not defined